In [1]:
'''
CS585_Lab3
CS585 Image and Video Computing
Lab 3
--------------
This program introduces the following concepts:
	a) Reading a stream of images from a webcamera, and displaying the video
	b) Skin color detection
	c) Background differencing
	d) Visualizing motion history
--------------
'''
import cv2
import sys
import numpy as np

In [2]:
a = np.random.random((5,5))
print(a)

[[0.65679911 0.28568062 0.5184591  0.96625137 0.96869598]
 [0.0025177  0.35795021 0.14399882 0.57121736 0.98288816]
 [0.98832649 0.38583503 0.7027842  0.23269736 0.7867104 ]
 [0.73782223 0.76786485 0.41981379 0.53527038 0.51659528]
 [0.96966022 0.18796745 0.62474262 0.45575449 0.96113966]]


In [3]:
mask = a > 0.5
print(mask.all() or mask.all())

False


In [23]:
def my_skin_detect(src):
    '''
    Function that detects whether a pixel belongs to the skin based on RGB values
    Args: 
        src The source color image
    Returns: 
        dst The destination grayscale image where skin pixels are colored white and the rest are colored black
    Surveys of skin color modeling and detection techniques:
    Vezhnevets, Vladimir, Vassili Sazonov, and Alla Andreeva. "A survey on pixel-based skin color detection techniques." Proc. Graphicon. Vol. 3. 2003.
    Kakumanu, Praveen, Sokratis Makrogiannis, and Nikolaos Bourbakis. "A survey of skin-color modeling and detection methods." Pattern recognition 40.3 (2007): 1106-1122.
    '''
    hsv_min = np.array([0, 60, 50], dtype = "uint8")
    hsv_max = np.array([30, 255, 255], dtype = "uint8")
    hsv_src = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)
    hsv_skin_region = cv2.inRange(hsv_src, hsv_min, hsv_max)
    
    dst = cv2.bitwise_and(src, src, mask = hsv_skin_region)
    
    return cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)


def my_frame_differencing(prev, curr):
    '''
    Function that does frame differencing between the current frame and the previous frame
    Args:
        src The current color image
        prev The previous color image
    Returns:
        dst The destination grayscale image where pixels are colored white if the corresponding pixel intensities in the current
    and previous image are not the same
    '''
    
    dst = cv2.absdiff(prev, curr)
    gs = cv2.cvtColor(dst, cv2.COLOR_BGR2GRAY)
    return (gs > 50).astype(np.uint8) * 255

def my_motion_energy(mh):
    '''
    Function that accumulates the frame differences for a certain number of pairs of frames
    Args:
        mh Vector of frame difference images
    Returns:
        dst The destination grayscale image to store the accumulation of the frame difference images
    '''
    return cv2.add(mh[0], mh[1])

In [ ]:
# ----------------
# a) Reading a stream of images from a webcamera, and displaying the video
# ----------------
# For more information on reading and writing video: http://docs.opencv.org/modules/highgui/doc/reading_and_writing_images_and_video.html
# open the video camera no. 0
cap = cv2.VideoCapture(0)

# if not successful, exit program
if not cap.isOpened():
    print("Cannot open the video cam")
    sys.exit()

# create a window called "MyVideo0"
cv2.namedWindow("Webcam", cv2.WINDOW_AUTOSIZE)

# read a new frame from video
ret, frame0 = cap.read()
if not ret:
    print("Cannot read a frame from video stream")

# show the frame in "Webcam" window
cv2.imshow("Webcam", frame0)

# create windows
cv2.namedWindow("MyVideo", cv2.WINDOW_AUTOSIZE)
cv2.namedWindow("FrameDiff", cv2.WINDOW_AUTOSIZE)
cv2.namedWindow("MotionEnergy", cv2.WINDOW_AUTOSIZE)

my_motion_history = []
fMH1 = np.zeros((np.shape(frame0)[0], np.shape(frame0)[1]), dtype=np.uint8)
fMH2 = np.zeros((np.shape(frame0)[0], np.shape(frame0)[1]), dtype=np.uint8)
fMH3 = np.zeros((np.shape(frame0)[0], np.shape(frame0)[1]), dtype=np.uint8)
my_motion_history.append(fMH1)
my_motion_history.append(fMH2)
my_motion_history.append(fMH3)

while(1):
    # read a new frame from video
    ret, frame = cap.read()
    # if not successful, break loop
    if not ret:
        print("Cannot read a frame from video stream")
        break

    cv2.imshow("Webcam", frame)

    # ----------------
    # b) Skin color detection
    # ----------------
    frame_dst = my_skin_detect(frame)
    cv2.imshow("Skin", frame_dst)

    # ----------------
    # c) Background differencing
    # ----------------

    # call my_frame_differencing function
    frame_dst = my_frame_differencing(frame0, frame)
    cv2.imshow("FrameDiff", frame_dst)

    # ----------------
    #  d) Visualizing motion history
    # ----------------
    
    my_motion_history.pop(0)
    my_motion_history.append(frame_dst)

    # call my_motion_energy function
    myMH = my_motion_energy(my_motion_history)
    cv2.imshow("MotionEnergy", myMH)
    frame0 = frame

    # wait for 'esc' key press for 30ms. If 'esc' key is pressed, break loop
    if cv2.waitKey(30) == 27:
        print("esc key is pressed by user")
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)